In [2]:
import h5py, numpy as np, geopandas as gpd
from shapely.geometry import LineString
from shapely.ops import unary_union
from shapely import wkb
from shapely.affinity import scale
from shapely import ops
from shapely.geometry import Polygon
from shapely import wkt

# ---- USER INPUTS ----
lat1, lon1 = -3.276522910174481, -59.85548776356948
lat2, lon2 = -3.274225103078294, -59.85389804800792
buffer_m   = 15                      # half-width of road in meters (tune!)
geom_h5    = "/insar-data/Curaca/miaplpy/network_delaunay_4/inputs/geometryRadar.h5"       # radar-grid geometry H5 (lat/lon arrays)
lat_key    = "latitude"               # dataset name for latitude
lon_key    = "longitude"              # dataset name for longitude
out_h5     = "road_roi_mask.h5"       # output mask (1=keep, 0=drop)
# ----------------------

# 1) Build a 2-point line in WGS84 (EPSG:4326)
line_wgs84 = LineString([(lon1, lat1), (lon2, lat2)])  # NOTE: (lon, lat) order

# 2) Buffer in meters: reproject to local UTM, buffer, reproject back to WGS84
gdf = gpd.GeoDataFrame(geometry=[line_wgs84], crs=4326)
utm = gdf.estimate_utm_crs()          # picks a sensible UTM zone for the line
gdf_utm = gdf.to_crs(utm)

# Flat ends (cap_style=2) are nice for straight road segments; use 1 for round
poly_utm = gdf_utm.buffer(buffer_m, cap_style=2).unary_union
poly_wgs84 = gpd.GeoSeries([poly_utm], crs=utm).to_crs(4326).iloc[0]

# 3) Load radar-grid lat/lon arrays (same shape as your processing grid)
with h5py.File(geom_h5, "r") as f:
    lat = f[lat_key][:]
    lon = f[lon_key][:]

ny, nx = lat.shape
assert lon.shape == (ny, nx), "lat/lon shape mismatch"

# 4) Vectorized point-in-polygon: True where pixel center falls inside road buffer
try:
    from shapely.vectorized import contains
    mask_bool = contains(poly_wgs84, lon, lat)   # (ny, nx) boolean array
except Exception:
    # Fallback using matplotlib.path if shapely.vectorized is unavailable
    import matplotlib.path as mpltPath
    # Get polygon exterior ring (in lon/lat)
    x, y = np.array(poly_wgs84.exterior.coords).T
    path = mpltPath.Path(np.column_stack([x, y]))
    mask_bool = path.contains_points(np.column_stack([lon.ravel(), lat.ravel()])).reshape(ny, nx)

mask = mask_bool.astype("uint8")  # 1=keep, 0=drop

# 5) Save HDF5 mask alongside shape-compatible dataset name "mask"
with h5py.File(out_h5, "w") as f:
    f.create_dataset("mask", data=mask, compression="gzip")

print(f"Saved {out_h5} with shape {mask.shape}, keep_fraction={mask.mean():.4f}")


Saved road_roi_mask.h5 with shape (28, 53), keep_fraction=0.1260


In [3]:
import h5py, numpy as np

path = "road_roi_mask.h5"   # your mask file
dset_name = "mask"          # dataset we wrote earlier

with h5py.File(path, "r+") as f:
    d = f[dset_name]
    ny, nx = d.shape

    # Put attrs on BOTH the file root and the dataset (some readers check one or the other)
    for obj in (f, d):
        obj.attrs["WIDTH"]  = np.int32(nx)
        obj.attrs["LENGTH"] = np.int32(ny)
        obj.attrs["DATA_TYPE"] = "uint8"
        obj.attrs["FILE_TYPE"] = "mask"
        # Workaround for versions that accidentally look for WIDHT (typo)
        obj.attrs["WIDHT"] = np.int32(nx)

print("Patched HDF5 attrs: WIDTH, LENGTH (and WIDHT) set.")

Patched HDF5 attrs: WIDTH, LENGTH (and WIDHT) set.
